In [1]:
from model import AttentionModel

In [2]:
keyPoints = {
    'NOSE': 0,
    'LEFT_EYE': 1,
    'RIGHT_EYE': 2,
    'LEFT_EAR':3,
    'RIGHT_EAR': 4,
    'LEFT_SHOULDER': 5,
    'RIGHT_SHOULDER':6,
    'LEFT_ELBOW': 7,
    'RIGHT_ELBOW': 8,
    'LEFT_WRIST': 9,
    'RIGHT_WRIST': 10,
    'LEFT_HIP': 11,
    'RIGHT_HIP': 12,
    'LEFT_KNEE': 13,
    'RIGHT_KNEE': 14,
    'LEFT_ANKLE': 15,
    'RIGHT_ANKLE': 16,
}

In [3]:
OUT_PATH = 'result/model_output_video.avi' # your video path for saving
DATA_PATH = 'Анализ бригад (телефон)/Есть телефон/00_26_30.mp4' # your video path for processing
SAVE_IMG_PATH = 'result'

In [4]:
model = AttentionModel('models/yolov8l_ep25.pt')
model.load_models()

In [5]:
model.detected_data

,Frame_id,Time,Objects_class,Position


In [6]:
model.process_video(DATA_PATH, OUT_PATH, pos_estimation=True, detection=True, save=True)


0: 384x640 2 persons, 55.1ms
Speed: 1.1ms preprocess, 55.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 60.1ms
Speed: 1.2ms preprocess, 60.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 53.0ms
Speed: 1.2ms preprocess, 53.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 52.3ms
Speed: 1.2ms preprocess, 52.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 52.5ms
Speed: 1.2ms preprocess, 52.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 52.8ms
Speed: 1.1ms preprocess, 52.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 48.2ms
Speed: 1.2ms preprocess, 48.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 53.8ms
Speed: 1.2ms preprocess, 53.8ms inference, 4.0ms postprocess per image at shape (

Time: 608.1052331924438


In [7]:
model.pos_est_data.to_csv('result/pos_est_data.csv', index=0)

In [8]:
model.detected_data.to_csv('result/detected_data.csv', index=0)